In [112]:
import keyring
from pprint import pprint

api_key_irp = keyring.get_password("irp_api_key", "birdblues")
api_secret_irp = keyring.get_password("irp_api_secret", "birdblues")

api_key_isa = keyring.get_password("isa_api_key", "birdblues")
api_secret_isa = keyring.get_password("isa_api_secret", "birdblues")

api_key_0 = keyring.get_password("api_key_0", "birdblues")
api_secret_0 = keyring.get_password("api_secret_0","birdblues")

api_key_1 = keyring.get_password("api_key_1", "birdblues")
api_secret_1 = keyring.get_password("api_secret_1","birdblues")

api_key_2 = keyring.get_password("api_key_2", "birdblues")
api_secret_2 = keyring.get_password("api_secret_2","birdblues")

# print(f"api_key_irp = {api_key_irp}")
# print(f"api_secret_irp = {api_secret_irp}")
# print(f"api_key_isa = {api_key_isa}")
# print(f"api_secret_isa = {api_secret_isa}")
# print(f"api_key_0 = {api_key_0}")
# print(f"api_secret_0 = {api_secret_0}")
# print(f"api_key_1 = {api_key_1}")
# print(f"api_secret_1 = {api_secret_1}")
# print(f"api_key_2 = {api_key_2}")
# print(f"api_secret_2 = {api_secret_2}")

In [113]:
acc_no_0 = "63981494-01" # 위탁
acc_no_1 = "64267316-22" # 연금저축 1
acc_no_2 = "64012548-22" # 연금저축 2
acc_no_isa = "64043278-01" # ISA
acc_no_irp = "64012548-29" # IR

In [114]:
import keyring

token_0 = keyring.get_password("access_token_0", "birdblues")
token_1 = keyring.get_password("access_token_1", "birdblues")
token_2 = keyring.get_password("access_token_2", "birdblues")
token_3 = keyring.get_password("access_token_3", "birdblues")
token_4 = keyring.get_password("access_token_4", "birdblues")

# print(f"token_0 = {token_0}")
# print(f"token_1 = {token_1}")
# print(f"token_2 = {token_2}")
# print(f"token_3 = {token_3}")
# print(f"token_4 = {token_4}")

In [115]:
import requests
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

column_mapping = {
    'trad_dt': '거래일',
    'pdno': '코드',
    'prdt_name': '상품명',
    'trad_dvsn_name': '거래구분',
    'loan_dt': '대출일',
    'hldg_qty': '보유',
    'pchs_unpr': '매입가격',
    'buy_qty': '매수수량',
    'buy_amt': '매수금액',
    'sll_pric': '매도가격',
    'sll_qty': '매도수량',
    'sll_amt': '매도금액',
    'rlzt_pfls': '실현손익',
    'pfls_rt': '손익률',
    'fee': '수수료',
    'tl_tax': '세금',
    'loan_int': '대출이자'
}

cols_to_convert = ['보유', '매입가격', '매수수량', '매수금액', '매도가격', '매도수량', '매도금액', '실현손익', '손익률', '수수료', '세금', '대출이자']

def get_trade_profit(access_token, app_key, app_secret, acc_no):
    
    output1_list = []
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/inquire-period-trade-profit"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC8715R"
        # "tr_cont": "",
        # "custtype": "P"
        
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "SORT_DVSN": "01",
        "PDNO": "",
        "INQR_STRT_DT": "20230101",
        "INQR_END_DT": "20241231",
        "CBLC_DVSN": "00",
        "CTX_AREA_NK100": "",
        "CTX_AREA_FK100": ""
    }
    res = requests.get(url, headers=headers, params=params)
    body = res.json()
    if 'output1' in body:
        output1_list.extend(body['output1'])
        
    # display(body)
    # print(res.headers['tr_cont'])

    while res.headers['tr_cont'] == "M":
        headers["tr_cont"] = res.headers['tr_cont']
        params["INQR_STRT_DT"] = body['ctx_area_nk100'][:8]
        params["CTX_AREA_NK100"] = body['ctx_area_nk100']
        params["CTX_AREA_FK100"] = body['ctx_area_fk100']
        res = requests.get(url, headers=headers, params=params)
        body = res.json()
        if 'output1' in body:
            output1_list.extend(body['output1'])
        # display(body)
        # print(res.headers['tr_cont'])
    df = pd.DataFrame(output1_list)
    df.rename(columns=column_mapping, inplace=True)
 
    # 각 컬럼에 대해서 쉼표 제거하고 숫자형으로 변환
    for col in cols_to_convert:
        df[col] = df[col].str.replace(',', '')  # 쉼표 제거
        df[col] = pd.to_numeric(df[col], errors='coerce')  # 숫자형으로 변환 (에러가 발생하면 NaN으로 처리)
        
    df['계좌'] = acc_no
    df['정산금'] = df['매도금액'] - df['매수금액'] - df['수수료'] - df['세금'] - df['대출이자']

    df['거래일'] = pd.to_datetime(df['거래일'], format='%Y%m%d')
    # df = df[['거래일', '종류', '거래금', '수량', '정산금', '수수료', '세금', '상품명', '코드', '계좌']]
    df.set_index('거래일', inplace=True)
    df.sort_index(inplace=True)
    df.drop_duplicates(inplace=True)
    
    return df

In [116]:
# access_token_0 = get_access_token(api_key_0, api_secret_0)
# access_token_1 = get_access_token(api_key_1, api_secret_1)
# access_token_2 = get_access_token(api_key_2, api_secret_2)
# access_token_3 = get_access_token(api_key_isa, api_secret_isa)
# access_token_4 = get_access_token(api_key_irp, api_secret_irp)

df1 = get_trade_profit(token_0, api_key_0, api_secret_0, acc_no_0)
df2 = get_trade_profit(token_1, api_key_1, api_secret_1, acc_no_1)
df3 = get_trade_profit(token_2, api_key_2, api_secret_2, acc_no_2)
df4 = get_trade_profit(token_3, api_key_isa, api_secret_isa, acc_no_isa)
df5 = get_trade_profit(token_4, api_key_irp, api_secret_irp, acc_no_irp)

df = pd.concat([df1, df2, df3, df4, df5])
df.sort_index(inplace=True)
display(df)


,코드,상품명,거래구분,대출일,보유,매입가격,매수수량,매수금액,매도가격,매도수량,매도금액,실현손익,손익률,수수료,세금,대출이자,계좌,정산금
거래일,,,,,,,,,,,,,,,,,,
2023-12-22,148070,KOSEF 국고채10년,현금,,1,114685,1,114685,0,0,0,0,0.00,0,0,0,64012548-29,-114685
2023-12-22,195980,PLUS 신흥국MSCI(합성 H),현금,,6,8970,6,53820,0,0,0,0,0.00,0,0,0,64012548-29,-53820
2023-12-22,272580,TIGER 단기채권액티브,현금,,2,53095,2,106190,0,0,0,0,0.00,0,0,0,64012548-29,-106190
2023-12-22,284430,KODEX 200미국채혼합,현금,,23,12460,23,286580,0,0,0,0,0.00,0,0,0,64012548-29,-286580
2023-12-22,360750,TIGER 미국S&P500,현금,,11,15435,11,169785,0,0,0,0,0.00,0,0,0,64012548-29,-169785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-12,218420,KODEX 미국S&P500에너지(합성),현금,,36,13085,36,471060,0,0,0,-19,0.00,19,0,0,64012548-22,-471079
2024-09-12,218420,KODEX 미국S&P500에너지(합성),현금,,106,13350,7,91435,0,0,0,-3,0.00,3,0,0,64043278-01,-91438
2024-09-12,475380,RISE 글로벌리얼티인컴,현금,,0,10505,0,0,10845,4,43380,1359,3.23,1,0,0,64012548-22,43379


In [117]:
# '코드'를 기준으로 데이터를 나누고 MultiIndex 컬럼 생성
def clean_code(df):
    df = df.pivot_table(index=['거래일'], columns='코드', values=['종류', '거래금', '수량', '정산금', '수수료', '세금', '상품명', '계좌'], aggfunc='first')
    df['상품명'] = df['상품명'].apply(lambda col: col.ffill().bfill())
    df['계좌'] = df['계좌'].apply(lambda col: col.ffill().bfill())
    df['종류'] = df['종류'].apply(lambda col: col.ffill().bfill())
    df.fillna(0, inplace=True)

    new_order = ['종류', '거래금', '수량', '정산금', '수수료', '세금', '상품명', '계좌']
    df = df.reorder_levels([1, 0], axis=1).sort_index(axis=1)
    df = df.reindex(columns=pd.MultiIndex.from_product([df.columns.levels[0], new_order]))
    
    return df 

# 주식 = clean_code(df)
# display(주식.columns.levels[0])

# for code in 주식.columns.levels[0]:
#     display(주식[code])

# 매수 = df[df['종류'] == '거래소매수']
# 매도 = df[df['종류'] == '거래소매도']

# display(매수)

# 결과 확인
# df_114800 = 주식.loc[:, ('114800', slice(None))]
# display(df_114800)

# dd_114800 = df[df['코드'] == '114800']
# dd_114800

In [118]:
import keyring
import pandas as pd
from sqlalchemy import create_engine

password = keyring.get_password("mysql-rocketpig", "root")
user = 'root'
host = '220.76.59.96'
port = '3306'  # 기본적으로 MySQL은 3306 포트를 사용합니다
db = 'stock_db'

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8")

# 연결 테스트
with engine.connect() as connection:
    print("SQLAlchemy로 MySQL 연결 성공!")

SQLAlchemy로 MySQL 연결 성공!


In [119]:
df.to_sql('stock', engine, if_exists='replace')
# df_buy.to_sql('stock_buy', engine, if_exists='replace')
# df_sell.to_sql('stock_sell', engine, if_exists='replace')

611

In [120]:
query = "SELECT * FROM stock"
주식 = pd.read_sql(query, con=engine)
주식.set_index('거래일', inplace=True)
주식

,코드,상품명,거래구분,대출일,보유,매입가격,매수수량,매수금액,매도가격,매도수량,매도금액,실현손익,손익률,수수료,세금,대출이자,계좌,정산금
거래일,,,,,,,,,,,,,,,,,,
2023-12-22,148070,KOSEF 국고채10년,현금,,1,114685,1,114685,0,0,0,0,0.00,0,0,0,64012548-29,-114685
2023-12-22,195980,PLUS 신흥국MSCI(합성 H),현금,,6,8970,6,53820,0,0,0,0,0.00,0,0,0,64012548-29,-53820
2023-12-22,272580,TIGER 단기채권액티브,현금,,2,53095,2,106190,0,0,0,0,0.00,0,0,0,64012548-29,-106190
2023-12-22,284430,KODEX 200미국채혼합,현금,,23,12460,23,286580,0,0,0,0,0.00,0,0,0,64012548-29,-286580
2023-12-22,360750,TIGER 미국S&P500,현금,,11,15435,11,169785,0,0,0,0,0.00,0,0,0,64012548-29,-169785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-12,218420,KODEX 미국S&P500에너지(합성),현금,,36,13085,36,471060,0,0,0,-19,0.00,19,0,0,64012548-22,-471079
2024-09-12,218420,KODEX 미국S&P500에너지(합성),현금,,106,13350,7,91435,0,0,0,-3,0.00,3,0,0,64043278-01,-91438
2024-09-12,475380,RISE 글로벌리얼티인컴,현금,,0,10505,0,0,10845,4,43380,1359,3.23,1,0,0,64012548-22,43379
